<a href="https://colab.research.google.com/github/kpetridis24/vf2-pp/blob/main/benchmark/profile_vf2pp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

VF2++ Profile Analysis

In [9]:
import networkx as nx
import collections
import random
!pip install memory_profiler
%load_ext memory_profiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 1.8 MB/s 
     |████████████████████████████████| 1.4 MB 15.2 MB/s 
  Attempting uninstall: aesara
    Found existing installation: aesara 2.7.9
    Uninstalling aesara-2.7.9:
      Successfully uninstalled aesara-2.7.9
  Attempting uninstall: aeppl
    Found existing installation: aeppl 0.0.33
    Uninstalling aeppl-0.0.33:
      Successfully uninstalled aeppl-0.0.33
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc 4.1.4 requires aeppl==0.0.33, but you have aeppl 0.0.35 which is incompatible.
pymc 4.1.4 requires aesara==2.7.9, but you have aesara 2.8.2 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simpl

## VF2++ ISO solver

In [10]:
_GraphParameters = collections.namedtuple(
    "_GraphParameters",
    [
        "G1",
        "G2",
        "G1_labels",
        "G2_labels",
        "nodes_of_G1Labels",
        "nodes_of_G2Labels",
        "G2_nodes_of_degree",
    ],
)

_StateParameters = collections.namedtuple(
    "_StateParameters",
    ["mapping", "reverse_mapping", "T1", "T1_tilde", "T2", "T2_tilde"],
)


def vf2pp_is_isomorphic(G1, G2, node_labels=None, default_label=None):
    if vf2pp_all_mappings(G1, G2, node_labels, default_label) is not None:
        return True
    return False


def vf2pp_all_mappings(G1, G2, node_labels=None, default_label=None):
    if G1.number_of_nodes() == 0 or G2.number_of_nodes() == 0:
        return False

    # Check that both graphs have the same number of nodes and degree sequence
    if not nx.faster_could_be_isomorphic(G1, G2):
        return False

    # Initialize parameters and cache necessary information about degree and labels
    graph_params, state_params = _initialize_parameters(
        G1, G2, node_labels, default_label
    )

    # Check if G1 and G2 have the same labels, and that number of nodes per label is equal between the two graphs
    if not _precheck_label_properties(graph_params):
        return False

    # Calculate the optimal node ordering
    node_order = _matching_order(graph_params)

    # Initialize the stack
    stack = []
    candidates = iter(_find_candidates(node_order[0], graph_params, state_params))
    stack.append((node_order[0], candidates))

    mapping = state_params.mapping
    reverse_mapping = state_params.reverse_mapping

    # Index of the node from the order, currently being examined
    matching_node = 1

    while stack:
        current_node, candidate_nodes = stack[-1]

        try:
            candidate = next(candidate_nodes)
        except StopIteration:
            # If no remaining candidates, return to a previous state, and follow another branch
            stack.pop()
            matching_node -= 1
            if stack:
                # Pop the previously added u-v pair, and look for a different candidate _v for u
                popped_node1, _ = stack[-1]
                popped_node2 = mapping[popped_node1]
                mapping.pop(popped_node1)
                reverse_mapping.pop(popped_node2)
                _restore_Tinout(popped_node1, popped_node2, graph_params, state_params)
            continue

        if _feasibility(current_node, candidate, graph_params, state_params):
            # Terminate if mapping is extended to its full
            if len(mapping) == G2.number_of_nodes() - 1:
                cp_mapping = mapping.copy()
                cp_mapping[current_node] = candidate
                return cp_mapping

            # Feasibility rules pass, so extend the mapping and update the parameters
            mapping[current_node] = candidate
            reverse_mapping[candidate] = current_node
            _update_Tinout(current_node, candidate, graph_params, state_params)
            # Append the next node and its candidates to the stack
            candidates = iter(
                _find_candidates(node_order[matching_node], graph_params, state_params)
            )
            stack.append((node_order[matching_node], candidates))
            matching_node += 1


def _precheck_label_properties(graph_params):
    G1, G2, G1_labels, G2_labels, nodes_of_G1Labels, nodes_of_G2Labels, _ = graph_params
    if any(
        label not in nodes_of_G1Labels or len(nodes_of_G1Labels[label]) != len(nodes)
        for label, nodes in nodes_of_G2Labels.items()
    ):
        return False
    return True


def _initialize_parameters(G1, G2, node_labels=None, default_label=-1):
    G1_labels = dict(G1.nodes(data=node_labels, default=default_label))
    G2_labels = dict(G2.nodes(data=node_labels, default=default_label))

    graph_params = _GraphParameters(
        G1,
        G2,
        G1_labels,
        G2_labels,
        nx.utils.groups(G1_labels),
        nx.utils.groups(G2_labels),
        nx.utils.groups({node: degree for node, degree in G2.degree()}),
    )

    state_params = _StateParameters(
        dict(), dict(), set(), set(G1.nodes()), set(), set(G2.nodes())
    )

    return graph_params, state_params


### Matching Node Ordering

In [18]:
import networkx as nx


def _matching_order(graph_params):
    G1, G2, G1_labels, _, _, nodes_of_G2Labels, _ = graph_params
    if not G1 and not G2:
        return {}

    V1_unordered = set(G1.nodes())
    label_rarity = {label: len(nodes) for label, nodes in nodes_of_G2Labels.items()}
    used_degrees = {node: 0 for node in G1}
    node_order = []

    while V1_unordered:
        rarest_nodes = _all_argmax(
            V1_unordered, key_function=lambda x: -label_rarity[G1_labels[x]]
        )
        max_node = max(rarest_nodes, key=G1.degree)

        for dlevel_nodes in bfs_layers(G1, max_node):
            while dlevel_nodes:
                max_used_deg_nodes = _all_argmax(
                    dlevel_nodes, key_function=lambda x: used_degrees[x]
                )
                max_deg_nodes = _all_argmax(
                    max_used_deg_nodes, key_function=lambda x: G1.degree[x]
                )
                next_node = min(max_deg_nodes, key=lambda x: label_rarity[G1_labels[x]])

                node_order.append(next_node)
                for node in G1.neighbors(next_node):
                    used_degrees[node] += 1

                dlevel_nodes.remove(next_node)
                label_rarity[G1_labels[next_node]] -= 1
                V1_unordered.discard(next_node)

    return node_order


def _all_argmax(nodes, key_function):
    best_nodes = []
    best = -float("inf")
    for n in nodes:
        if key_function(n) > best:
            best = key_function(n)
            best_nodes = [n]
            continue
        if key_function(n) == best:
            best_nodes.append(n)

    return best_nodes


### Candidate Selection

In [12]:
def _find_candidates(u, graph_params, state_params):
    G1, G2, G1_labels, _, _, nodes_of_G2Labels, G2_nodes_of_degree = graph_params
    mapping, reverse_mapping, _, _, _, T2_out = state_params

    covered_neighbors = [nbr for nbr in G1[u] if nbr in mapping]
    if not covered_neighbors:
        candidates = set(nodes_of_G2Labels[G1_labels[u]])
        candidates.intersection_update(G2_nodes_of_degree[G1.degree[u]])
        candidates.intersection_update(T2_out)
        candidates.difference_update(reverse_mapping)
        candidates.difference_update(
            {
                node
                for node in candidates
                if G1.number_of_edges(u, u) != G2.number_of_edges(node, node)
            }
        )
        return candidates

    nbr1 = covered_neighbors[0]
    common_nodes = set(G2[mapping[nbr1]])

    for nbr1 in covered_neighbors[1:]:
        common_nodes.intersection_update(G2[mapping[nbr1]])

    common_nodes.difference_update(reverse_mapping)
    common_nodes.intersection_update(G2_nodes_of_degree[G1.degree[u]])
    common_nodes.intersection_update(nodes_of_G2Labels[G1_labels[u]])
    common_nodes.difference_update(
        {
            node
            for node in common_nodes
            if G1.number_of_edges(u, u) != G2.number_of_edges(node, node)
        }
    )
    return common_nodes


### Feasibility Rules 

In [13]:
import networkx as nx


def _feasibility(node1, node2, graph_params, state_params):
    G1 = graph_params.G1

    if _cut_PT(node1, node2, graph_params, state_params):
        return False

    if G1.is_multigraph():
        if not _consistent_PT(node1, node2, graph_params, state_params):
            return False

    return True


def _cut_PT(u, v, graph_params, state_params):
    G1, G2, G1_labels, G2_labels, _, _, _ = graph_params
    _, _, T1, T1_tilde, T2, T2_tilde = state_params

    u_labels_neighbors = nx.utils.groups({n1: G1_labels[n1] for n1 in G1[u]})
    v_labels_neighbors = nx.utils.groups({n2: G2_labels[n2] for n2 in G2[v]})

    # if the neighbors of u, do not have the same labels as those of v, NOT feasible.
    if set(u_labels_neighbors.keys()) != set(v_labels_neighbors.keys()):
        return True

    for label, G1_nbh in u_labels_neighbors.items():
        G2_nbh = v_labels_neighbors[label]

        if isinstance(G1, nx.MultiGraph):
            # Check for every neighbor in the neighborhood, if u-nbr1 has same edges as v-nbr2
            u_nbrs_edges = sorted(G1.number_of_edges(u, x) for x in G1_nbh)
            v_nbrs_edges = sorted(G2.number_of_edges(v, x) for x in G2_nbh)
            if any(
                u_nbr_edges != v_nbr_edges
                for u_nbr_edges, v_nbr_edges in zip(u_nbrs_edges, v_nbrs_edges)
            ):
                return True

        if len(T1.intersection(G1_nbh)) != len(T2.intersection(G2_nbh)):
            return True
        if len(T1_tilde.intersection(G1_nbh)) != len(T2_tilde.intersection(G2_nbh)):
            return True

    return False


def _consistent_PT(u, v, graph_params, state_params):
    G1, G2 = graph_params.G1, graph_params.G2
    mapping, reverse_mapping = state_params.mapping, state_params.reverse_mapping

    for neighbor in G1[u]:
        if neighbor in mapping:
            if G1.number_of_edges(u, neighbor) != G2.number_of_edges(
                v, mapping[neighbor]
            ):
                return False

    for neighbor in G2[v]:
        if neighbor in reverse_mapping:
            if G1.number_of_edges(u, reverse_mapping[neighbor]) != G2.number_of_edges(
                v, neighbor
            ):
                return False
    return True


### State Updating/Restoring

In [14]:
def _update_Tinout(new_node1, new_node2, graph_params, state_params):
    G1, G2, _, _, _, _, _ = graph_params
    mapping, reverse_mapping, T1, T1_tilde, T2, T2_tilde = state_params

    uncovered_neighbors_G1 = {nbr for nbr in G1[new_node1] if nbr not in mapping}
    uncovered_neighbors_G2 = {
        nbr for nbr in G2[new_node2] if nbr not in reverse_mapping
    }

    # Add the uncovered neighbors of node1 and node2 in T1 and T2 respectively
    T1.update(uncovered_neighbors_G1)
    T2.update(uncovered_neighbors_G2)
    T1.discard(new_node1)
    T2.discard(new_node2)

    T1_tilde.difference_update(uncovered_neighbors_G1)
    T2_tilde.difference_update(uncovered_neighbors_G2)
    T1_tilde.discard(new_node1)
    T2_tilde.discard(new_node2)


def _restore_Tinout(popped_node1, popped_node2, graph_params, state_params):
    # If the node we want to remove from the mapping, has at least one covered neighbor, add it to T1.
    G1, G2, _, _, _, _, _ = graph_params
    mapping, reverse_mapping, T1, T1_out, T2, T2_out = state_params

    is_added = False
    for nbr in G1[popped_node1]:
        if nbr in mapping:
            T1.add(
                popped_node1
            )  # if a neighbor of the excluded node1 is in the mapping, keep node1 in T1
            is_added = True
        else:  # check if its neighbor has another connection with a covered node. If not, only then exclude it from T1
            if any(nbr2 in mapping for nbr2 in G1[nbr]):
                continue
            T1.discard(nbr)
            T1_out.add(nbr)

    # Case where the node is not present in neither the mapping nor T1. By deffinition it should belong to T1_out
    if not is_added:
        T1_out.add(popped_node1)

    is_added = False
    for nbr in G2[popped_node2]:
        if nbr in reverse_mapping:
            T2.add(popped_node2)
            is_added = True
        else:
            if any(nbr2 in reverse_mapping for nbr2 in G2[nbr]):
                continue
            T2.discard(nbr)
            T2_out.add(nbr)

    if not is_added:
        T2_out.add(popped_node2)


## Profile Analysis

### Initialization

In [15]:
import itertools

# Label values
colors = [
    "white",
    "black",
    "green",
    "purple",
    "orange",
    "red",
    "blue",
    "pink",
    "yellow",
    "none",
]

# Create Graphs
G1 = nx.gnp_random_graph(450, 0.6, 42)
G2 = nx.gnp_random_graph(450, 0.6, 42)

# Assign Labels 
nx.set_node_attributes(G1, dict(zip(G1, itertools.cycle(colors))), "label")
nx.set_node_attributes(G2, dict(zip(G1, itertools.cycle(colors))), "label")

In [17]:
def bfs_layers(G, sources):
    if sources in G:
        sources = [sources]

    current_layer = list(sources)
    visited = set(sources)

    for source in current_layer:
        if source not in G:
            raise nx.NetworkXError(f"The node {source} is not in the graph.")

    # this is basically BFS, except that the current layer only stores the nodes at
    # same distance from sources at each iteration
    while current_layer:
        yield current_layer
        next_layer = list()
        for node in current_layer:
            for child in G[node]:
                if child not in visited:
                    visited.add(child)
                    next_layer.append(child)
        current_layer = next_layer

### CPU utilization

In [19]:
%timeit vf2pp_is_isomorphic(G1, G2, node_labels=None)

442 ms ± 3.84 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%timeit nx.is_isomorphic(G1, G2)

### Memory consumption

In [20]:
%memit vf2pp_is_isomorphic(G1, G2, node_labels=None)

peak memory: 169.55 MiB, increment: 0.12 MiB


In [21]:
%memit nx.is_isomorphic(G1, G2)

peak memory: 169.91 MiB, increment: 0.36 MiB


### Stats

In [22]:
%prun vf2pp_is_isomorphic(G1, G2, node_labels=None)